In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:


# reading raw data
df = pd.read_csv('Iran.Khodro.csv.txt', parse_dates=['<DTYYYYMMDD>'])
close_values = df['<CLOSE>'].to_numpy()

# reshaping close_values
close_values = np.reshape(close_values, (-1, 1))

# scaling the close_values into (0, 1)
sc = MinMaxScaler(feature_range=(-1, 1))
# sc = StandardScaler()
close_values = sc.fit_transform(close_values)

# print(sc.mean_, sc.scale_)
# fliping close_values to be in the right order of time from past to future
close_values = np.flipud(close_values)

# applying short-term memory filter of time-lag P and future Q to close_values
P = 15
Q = 30
memory_response = series_to_supervised(close_values, n_in=P, n_out=Q)
# print(memory_response)
memory_response = memory_response.to_numpy()

# memory_response = np.reshape(memory_response, (-1, P, 1))


# constructing training and test sets
x_train = memory_response[:-30, :-Q]
y_train = memory_response[:-30, -Q:]


x_test = memory_response[-30:, :-Q]
y_test = memory_response[-30:, -Q:]


print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
# initializing feedforward neural network model
tf.keras.backend.clear_session()

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='./checkpoints/FFNN_2layer_e100_b1_p15.hdf5',
    monitor='val_loss',
    save_best_only=True
)
regressor = tf.keras.Sequential([
    tf.keras.layers.Dense(P, activation='relu', input_dim=P),
    # tf.keras.layers.Dense(P, activation='relu'),
    # tf.keras.layers.Dense(P, activation='relu'),
    tf.keras.layers.Dense(Q),
])

# compiling the model
regressor.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# fitting the feedforward neural network to traing data
# history = regressor.fit(x_train,
#                         y_train,
#                         epochs=100,
#                         batch_size=1,
#                         validation_data=(x_test, y_test),
#                         callbacks=[checkpoint]
#                         )

In [ ]:
regressor.load_weights('./checkpoints/FFNN_2layer_e100_b1_p15.hdf5')


predictions = []
# memory_buffer = y_train[-P//Q:]
memory_buffer = close_values[-30 - P:-30]
memory_buffer = memory_buffer.reshape((1, P))
print(memory_buffer.shape)


# memory_buffer = np.reshape(memory_buffer, (-1, P, 1))
# print(memory_buffer.shape)
predictions = regressor.predict(memory_buffer)
predictions = np.reshape(predictions, (-1))
print(predictions.shape)

# 5% rule:
for i in range(predictions.shape[0] - 1):
    delta_prediction = (predictions[i + 1] - predictions[i]) / predictions[i]
    if delta_prediction > 0.05:
        predictions[i+1] = predictions[i] * 1.05
    elif delta_prediction < -0.05:
        predictions[i+1] = predictions[i] * 0.95

# plotting results
print(close_values[-30:].T.reshape(30,))
plt.plot(close_values[-30:].T.reshape(30,), color='blue', label='observed values')
plt.plot(predictions, color='red', label='predicted values')
plt.show()
print(predictions.shape)

sess = tf.Session()
test_mse = sess.run(tf.losses.mean_squared_error(close_values[-30:].T.reshape(30,), predictions))
test_mae = mean_absolute_error(close_values[-30:].T.reshape(30,), predictions)
print('test MSE: ', test_mse)
print('test MAE: ', test_mae)




predictions = sc.inverse_transform(predictions.reshape(-1, 1))
print(predictions)